In [1]:
import torch
from datasets.datasets import SHHSdataset
from datasets.augmentations import *

In [2]:
ds = SHHSdataset(
    data_path='../../data/',
    first_patient=1,
    num_patients=3,
    exclude_test_set=(1,3)
)

In [4]:
ds.X1.size()

torch.Size([689, 1, 3000])

In [8]:
ds.X1.element_size()*ds.X1.nelement()

2214540000

In [21]:
from datasets.datamodules import EEGdataModule
import torch.utils.data as data
dl = data.DataLoader(
    ds,
    batch_size=8192,
    shuffle=True,
)


In [22]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [24]:
batch = next(iter(dl))
eegs, labels = batch[0].to(device), batch[1].to(device)

In [18]:
eegs.element_size()*eegs.nelement()

12288000

In [19]:
del eegs, labels

In [20]:
torch.cuda.empty_cache()

In [26]:
from models.simclr_model import SimCLR
from models.mymodules import CNN_head
import torch.nn as nn
model = SimCLR(
    encoder=CNN_head([32, 32, 64], 100),
    projector=nn.Linear(100,5),
    optim_hparams={
        'lr': 1e-4,
        'max_epochs': 10
    },
    temperature=1
)
model

C:\Users\tomsm\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
  rank_zero_warn(
C:\Users\tomsm\anaconda3\lib\site-packages\pytorch_lightning\utilities\parsing.py:268: UserWarning: Attribute 'projector' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['projector'])`.
  rank_zero_warn(


SimCLR(
  (f): CNN_head(
    (model): Sequential(
      (0): CNN_block(
        (net): Sequential(
          (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=same, bias=False)
          (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
      )
      (1): CNN_block(
        (net): Sequential(
          (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=same, bias=False)
          (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
      )
      (2): CNN_block(
        (net): Sequential(
          (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=same, bias=False)
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True

In [27]:
model.to(device)

SimCLR(
  (f): CNN_head(
    (model): Sequential(
      (0): CNN_block(
        (net): Sequential(
          (0): Conv1d(1, 32, kernel_size=(3,), stride=(1,), padding=same, bias=False)
          (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
      )
      (1): CNN_block(
        (net): Sequential(
          (0): Conv1d(32, 32, kernel_size=(3,), stride=(1,), padding=same, bias=False)
          (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
        )
      )
      (2): CNN_block(
        (net): Sequential(
          (0): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=same, bias=False)
          (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True